In [1]:
from pymongo import MongoClient

# Connect to your MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["myDatabase"]
collection = db["myCollection"]

# Fetch your scraped documents (assume stored under 'content' field)
documents = [doc['page_text'] for doc in collection.find()]


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chunk and Preprocess the data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = splitter.create_documents(documents)


In [3]:
docs

[Document(metadata={}, page_content='Buffer [LINK_1]  class torch.nn.parameter.Buffer( data=None , * , persistent=True ) [source]  [source]  [LINK_2]  A kind of Tensor that should not be considered a model\nparameter. For example, BatchNorm’s running_mean is not a parameter, but is part of the module’s state.  Buffers are Tensor subclasses, that have a\nvery special property when used with Module s – when they’re\nassigned as Module attributes they are automatically added to the list of'),
 Document(metadata={}, page_content='its buffers, and will appear e.g. in buffers() iterator.\nAssigning a Tensor doesn’t have such effect. One can still assign a Tensor as explicitly by using\nthe register_buffer() function.  Parameters  data ( Tensor ) – buffer tensor.  persistent ( bool  ,  optional ) – whether the buffer is part of the module’s state_dict . Default: True\n Next  Previous  © Copyright 2024, PyTorch Contributors.  Built with Sphinx using a theme provided by Read the Docs .'),
 Docu

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# Generate embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\kuber\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kuber\anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kuber\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

In [7]:
from langchain.vectorstores import FAISS

# Checkpoint for Vector Store
vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("faiss_index")


In [9]:
# Load saved Vector Store
vectorstore = FAISS.load_local("faiss_index", embedding_model,allow_dangerous_deserialization=True)